# Data loading

MSGPACK --> Parquet.

In [1]:
from pathlib import Path
from tqdm.auto import tqdm

import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import shapely
from matplotlib import pyplot as plt

from ship_routing.app.routing import RoutingResult, RoutingLog
import msgpack
from tqdm.auto import tqdm

import warnings

warnings.filterwarnings("ignore")

In [2]:
data_files = sorted(Path("results/").glob("results_1*.msgpack"))
len(data_files)

85

In [3]:
def load_results_raw(msgpack_file_list: list) -> dict:
    raw_results = {}
    for mf in msgpack_file_list:
        with open(mf, "rb") as f:
            raw_results.update(msgpack.unpack(f, raw=False))
    return raw_results

In [4]:
def load_result_for_key(key: str = None, msgpack_file_list: list = None):
    return RoutingResult.from_msgpack(load_results_raw(msgpack_file_list)[key])

In [5]:
def load_results(msgpack_file_list: list) -> dict[str, RoutingResult]:
    """Load all results from msgpack file list.

    Parameters
    ----------
    msgpack_file_list : list
        Paths to msgpack file containing serialized results.

    Returns
    -------
    dict[str, RoutingResult]
        Dictionary mapping result keys to RoutingResult objects.
    """
    raw_results = load_results_raw(msgpack_file_list)
    return {
        key: RoutingResult.from_msgpack(value)
        for key, value in tqdm(list(raw_results.items()), desc="records")
    }

In [6]:
results = load_results(data_files)
len(results)

records:   0%|          | 0/168471 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


168471

In [7]:
def get_journey_params_df(routing_results_dict: dict = None):
    def _fix_waypoints(dct):
        dct["lon_waypoints"] = str(dct["lon_waypoints"])
        dct["lat_waypoints"] = str(dct["lat_waypoints"])
        return dct

    df = pd.concat(
        [
            pd.DataFrame(
                _fix_waypoints(rr.logs.config["journey"]),
                index=[
                    f,
                ],
            )
            for f, rr in routing_results_dict.items()
        ]
    ).add_prefix("journey_")
    df = df.assign(
        journey_lon_waypoints=df["journey_lon_waypoints"].astype("category"),
        journey_lat_waypoints=df["journey_lat_waypoints"].astype("category"),
        journey_name=df["journey_name"].astype("category"),
        journey_time_start=df["journey_time_start"].astype("category"),
        # journey_time_end=df["journey_time_end"].astype("category"),
        journey_speed_knots=df["journey_speed_knots"].astype("category"),
    )
    df.index = df.index.rename("filename")
    return df

In [8]:
df_journey = get_journey_params_df(results)
df_journey

,journey_name,journey_lon_waypoints,journey_lat_waypoints,journey_time_start,journey_time_end,journey_speed_knots,journey_time_resolution_hours
filename,,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T16-52-02.421_55875002-ea2e-4cd2-9733-ab2634d0b5f8,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
...,...,...,...,...,...,...,...
result:2025-12-07T19-59-45.091_3796470d-20ff-427b-afc0-88583fa9c8ba,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0
result:2025-12-07T20-22-56.333_c8f5ebfc-48c9-4dc4-9862-294c1452039d,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0


In [9]:
def get_hyper_params_df(routing_results_dict: dict = None):
    df = pd.concat(
        [
            pd.DataFrame(
                rr.logs.config["hyper"],
                index=[
                    f,
                ],
            )
            for f, rr in routing_results_dict.items()
        ]
    ).add_prefix("hyper_")
    df = df.assign(
        hyper_crossover_strategy=df["hyper_crossover_strategy"].astype("category")
    )
    df.index = df.index.rename("filename")
    return df

In [10]:
df_params = pd.merge(
    get_hyper_params_df(results),
    get_journey_params_df(results),
    left_index=True,
    right_index=True,
)
df_params

,hyper_population_size,hyper_random_seed,hyper_selection_acceptance_rate_warmup,hyper_mutation_width_fraction_warmup,hyper_mutation_displacement_fraction_warmup,hyper_generations,hyper_offspring_size,hyper_crossover_rounds,hyper_selection_quantile,hyper_selection_acceptance_rate,...,hyper_distance_increment,hyper_num_workers,hyper_executor_type,journey_name,journey_lon_waypoints,journey_lat_waypoints,journey_time_start,journey_time_end,journey_speed_knots,journey_time_resolution_hours
filename,,,,,,,,,,,,,,,,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,32,202838879,0.3,0.5,0.10,4,32,0,0.10,0.00,...,10000.0,8,process,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,32,240864382,0.3,0.9,0.10,4,32,1,0.10,0.25,...,10000.0,8,process,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10000.0,8,process,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,32,3459840799,0.3,0.5,0.25,4,32,2,0.25,0.25,...,10000.0,8,process,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
result:2025-12-06T16-52-02.421_55875002-ea2e-4cd2-9733-ab2634d0b5f8,128,3410662920,0.3,0.9,0.10,4,128,1,0.25,0.25,...,10000.0,8,process,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-01-01T00:00:00,2021-01-14T17:00:51,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
result:2025-12-07T19-59-45.091_3796470d-20ff-427b-afc0-88583fa9c8ba,32,370734705,0.3,0.9,0.10,4,32,1,0.25,0.25,...,10000.0,8,process,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0
result:2025-12-07T20-22-56.333_c8f5ebfc-48c9-4dc4-9862-294c1452039d,128,3286005916,0.3,0.9,0.25,2,128,2,0.25,0.25,...,10000.0,8,process,Atlantic_forward,"[-80.5, -11.0]","[30.0, 50.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,128,1835893064,0.3,0.5,0.25,4,128,0,0.10,0.00,...,10000.0,8,process,Atlantic_backward,"[-11.0, -80.5]","[50.0, 30.0]",2021-11-01T00:00:00,2021-11-12T10:10:42,12.0,6.0


In [11]:
def get_runtime(routing_results_dict: dict = None):
    _records = []
    for f, rr in routing_results_dict.items():
        _records.extend(
            [
                {
                    "filename": f,
                    "runtime": rr.logs.to_dataframe().timestamp.max()
                    - rr.logs.to_dataframe().timestamp.min(),
                }
            ]
        )
    df = pd.DataFrame.from_records(_records).set_index("filename")
    df = df.assign(runtime_seconds=df.runtime.dt.total_seconds())
    return df

In [12]:
df_runtime = get_runtime(results)
df_runtime

,runtime,runtime_seconds
filename,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,0 days 00:01:16,76.0
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,0 days 00:01:37,97.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,0 days 00:04:02,242.0
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,0 days 00:03:33,213.0
result:2025-12-06T16-52-02.421_55875002-ea2e-4cd2-9733-ab2634d0b5f8,0 days 00:06:35,395.0
...,...,...
result:2025-12-07T19-59-45.091_3796470d-20ff-427b-afc0-88583fa9c8ba,0 days 00:01:48,108.0
result:2025-12-07T20-22-56.333_c8f5ebfc-48c9-4dc4-9862-294c1452039d,0 days 00:02:48,168.0
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,0 days 00:04:49,289.0


In [13]:
def get_elite(routing_results_dict: dict = None):
    _records = []
    for f, rr in tqdm(routing_results_dict.items()):
        seed_member = rr.seed_member
        _records.extend(
            [
                {
                    "filename": f,
                    "n_elite": n,
                    "elite_length_meters": m.route.length_meters,
                    "elite_length_relative": m.route.length_meters
                    / seed_member.route.length_meters,
                    "elite_cost_absolute": m.cost,
                    "elite_cost_relative": m.cost / seed_member.cost,
                    "geometry": m.route.line_string,
                }
                for n, m in enumerate(rr.elite_population.members)
            ]
        )
    return pd.DataFrame.from_records(_records).set_index("filename")

In [14]:
df_elite = get_elite(results)
df_elite

  0%|          | 0/168471 [00:00<?, ?it/s]

,n_elite,elite_length_meters,elite_length_relative,elite_cost_absolute,elite_cost_relative,geometry
filename,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099..."
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099..."
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,0,6.102691e+06,1.001534,9.154279e+12,0.948557,"LINESTRING (-11 50.00000000000001, -12.5311121..."
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,1,6.096694e+06,1.000550,9.457689e+12,0.979995,"LINESTRING (-11 50, -12.527629087397221 50.105..."
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,0,6.237164e+06,1.023603,1.018871e+13,0.000000,"LINESTRING (-80.5 29.99999999999999, -79.65863..."
...,...,...,...,...,...,...
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,0,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,1,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...
result:2025-12-07T20-11-44.075_c6ae1298-a99c-48fd-ad6f-ba982e260699,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-80.5 30, -79.47780384738617 30.80..."


In [15]:
df_merged = df_params.merge(
    df_elite, left_index=True, right_index=True, how="inner"
).merge(df_runtime, left_index=True, right_index=True, how="inner")
df_merged

,hyper_population_size,hyper_random_seed,hyper_selection_acceptance_rate_warmup,hyper_mutation_width_fraction_warmup,hyper_mutation_displacement_fraction_warmup,hyper_generations,hyper_offspring_size,hyper_crossover_rounds,hyper_selection_quantile,hyper_selection_acceptance_rate,...,journey_speed_knots,journey_time_resolution_hours,n_elite,elite_length_meters,elite_length_relative,elite_cost_absolute,elite_cost_relative,geometry,runtime,runtime_seconds
filename,,,,,,,,,,,,,,,,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,32,202838879,0.3,0.5,0.10,4,32,0,0.10,0.00,...,10.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099...",0 days 00:01:16,76.0
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,32,240864382,0.3,0.9,0.10,4,32,1,0.10,0.25,...,10.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099...",0 days 00:01:37,97.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,0,6.102691e+06,1.001534,9.154279e+12,0.948557,"LINESTRING (-11 50.00000000000001, -12.5311121...",0 days 00:04:02,242.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,1,6.096694e+06,1.000550,9.457689e+12,0.979995,"LINESTRING (-11 50, -12.527629087397221 50.105...",0 days 00:04:02,242.0
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,32,3459840799,0.3,0.5,0.25,4,32,2,0.25,0.25,...,10.0,6.0,0,6.237164e+06,1.023603,1.018871e+13,0.000000,"LINESTRING (-80.5 29.99999999999999, -79.65863...",0 days 00:03:33,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,128,1835893064,0.3,0.5,0.25,4,128,0,0.10,0.00,...,12.0,6.0,0,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...,0 days 00:04:49,289.0
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,128,1835893064,0.3,0.5,0.25,4,128,0,0.10,0.00,...,12.0,6.0,1,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...,0 days 00:04:49,289.0
result:2025-12-07T20-11-44.075_c6ae1298-a99c-48fd-ad6f-ba982e260699,128,3662113073,0.3,0.9,0.10,2,128,1,0.10,0.25,...,12.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-80.5 30, -79.47780384738617 30.80...",0 days 00:01:42,102.0


In [16]:
df_merged

,hyper_population_size,hyper_random_seed,hyper_selection_acceptance_rate_warmup,hyper_mutation_width_fraction_warmup,hyper_mutation_displacement_fraction_warmup,hyper_generations,hyper_offspring_size,hyper_crossover_rounds,hyper_selection_quantile,hyper_selection_acceptance_rate,...,journey_speed_knots,journey_time_resolution_hours,n_elite,elite_length_meters,elite_length_relative,elite_cost_absolute,elite_cost_relative,geometry,runtime,runtime_seconds
filename,,,,,,,,,,,,,,,,,,,,,
result:2025-12-06T17-46-22.757_c17b2c84-f6f5-4c24-86b1-576e05ce458f,32,202838879,0.3,0.5,0.10,4,32,0,0.10,0.00,...,10.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099...",0 days 00:01:16,76.0
result:2025-12-06T17-45-47.883_8a96049a-b448-4688-a683-7bc1ad2f08f1,32,240864382,0.3,0.9,0.10,4,32,1,0.10,0.25,...,10.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-11 50, -12.539095689445155 50.099...",0 days 00:01:37,97.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,0,6.102691e+06,1.001534,9.154279e+12,0.948557,"LINESTRING (-11 50.00000000000001, -12.5311121...",0 days 00:04:02,242.0
result:2025-12-06T17-44-58.180_8709a686-711d-4bf0-a5df-1d5eaf040a8d,128,1282251641,0.3,0.5,0.25,1,128,2,0.25,0.00,...,10.0,6.0,1,6.096694e+06,1.000550,9.457689e+12,0.979995,"LINESTRING (-11 50, -12.527629087397221 50.105...",0 days 00:04:02,242.0
result:2025-12-06T16-34-19.208_1ba20d20-34f7-44e0-b86b-61d0db88fe09,32,3459840799,0.3,0.5,0.25,4,32,2,0.25,0.25,...,10.0,6.0,0,6.237164e+06,1.023603,1.018871e+13,0.000000,"LINESTRING (-80.5 29.99999999999999, -79.65863...",0 days 00:03:33,213.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,128,1835893064,0.3,0.5,0.25,4,128,0,0.10,0.00,...,12.0,6.0,0,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...,0 days 00:04:49,289.0
result:2025-12-07T20-31-58.625_46e15cd9-a913-45f7-8281-cbaa2872ee10,128,1835893064,0.3,0.5,0.25,4,128,0,0.10,0.00,...,12.0,6.0,1,6.117856e+06,1.004023,8.579450e+12,0.911326,LINESTRING (-10.999999999999996 50.00000000000...,0 days 00:04:49,289.0
result:2025-12-07T20-11-44.075_c6ae1298-a99c-48fd-ad6f-ba982e260699,128,3662113073,0.3,0.9,0.10,2,128,1,0.10,0.25,...,12.0,6.0,0,6.093342e+06,1.000000,NaN,NaN,"LINESTRING (-80.5 30, -79.47780384738617 30.80...",0 days 00:01:42,102.0


In [17]:
gpd.GeoDataFrame(df_merged).to_parquet("results/results_prelim.pq")